In [ ]:
'''
def detect_n_plot(img_path):
    img = cv2.imread(img_path)
    height, width, channels = img.shape
    
    # Detecting Objects
    print("Detecting... please wait!")
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    
    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and class_id == 0:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
                
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    font = cv2.FONT_HERSHEY_PLAIN
    person_found = 0
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = (0, 255, 0)
            font_color = (0, 0, 255)
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y + 30), font, 1, font_color, 1)
   
    print(len(boxes))
    cv2.imshow("Image", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
'''

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import math

In [2]:
from itertools import chain 
from constants import *

In [3]:
yolo_path = 'yolo_pretrained'
SAFE_DISTANCE = 60
os.listdir(yolo_path)


['coco.names', 'yolov3.cfg', 'yolov3.weights']

# Load YOLO

In [5]:
print('Loading YOLO')
net = cv2.dnn.readNet("yolo_pretrained/yolov3.weights", "yolo_pretrained/yolov3.cfg")
LABELS = []
with open("yolo_pretrained/coco.names", "r") as f:
    LABELS = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
COLORS = np.random.uniform(0, 255, size=(len(LABELS), 3))

Loading YOLO


In [41]:
img = cv2.imread("test_images/people.jpg")
H, W, channels = img.shape
    
# Detecting Objects
print("Detecting... please wait!")
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
layer_outputs  = net.forward(output_layers)

Detecting... please wait!


In [42]:
boxes = []
confidences = []
classIDs = []
lines = []
box_centers = []

for output in layer_outputs:
    for detection in output:

        scores = detection[5:]
        classID = np.argmax(scores)
        confidence = scores[classID]

        if confidence > 0.5 and classID == 0:
            box = detection[0:4] * np.array([W, H, W, H])
            (centerX, centerY, width, height) = box.astype('int')
            x = int(centerX - (width / 2))
            y = int(centerY - (height / 2))
            box_centers = [centerX, centerY]
            boxes.append([x, y, int(width), int(height)])
            confidences.append(float(confidence))
            classIDs.append(classID)

In [43]:
idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.3)

if len(idxs) > 0:
    unsafe = []
    count = 0

    for i in idxs.flatten():

        (x, y) = (boxes[i][0], boxes[i][1])
        (w, h) = (boxes[i][2], boxes[i][3])
        centeriX = boxes[i][0] + (boxes[i][2] // 2)
        centeriY = boxes[i][1] + (boxes[i][3] // 2)

        color = [int(c) for c in COLORS[classIDs[i]]]
        text = '{}: {:.4f}'.format(LABELS[classIDs[i]], confidences[i])

        idxs_copy = list(idxs.flatten())
        idxs_copy.remove(i)

        for j in np.array(idxs_copy):
            centerjX = boxes[j][0] + (boxes[j][2] // 2)
            centerjY = boxes[j][1] + (boxes[j][3] // 2)

            distance = math.sqrt(math.pow(centerjX - centeriX, 2) + math.pow(centerjY - centeriY, 2))

            if distance <= SAFE_DISTANCE:
                cv2.line(img, (boxes[i][0] + (boxes[i][2] // 2), boxes[i][1]  + (boxes[i][3] // 2)), (boxes[j][0] + (boxes[j][2] // 2), boxes[j][1] + (boxes[j][3] // 2)), (0, 0, 255), 2)
                unsafe.append([centerjX, centerjY])
                unsafe.append([centeriX, centeriY])

        if centeriX in chain(*unsafe) and centeriY in chain(*unsafe):
            count += 1
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 1)
        else:
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 1)

        cv2.putText(img, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
        cv2.rectangle(img, (50, 50), (450, 90), (0, 0, 0), -1)
        cv2.putText(img, 'No. of people unsafe: {}'.format(count), (70, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 1)

In [44]:
# cv2.imwrite("detected.jpg", img)
cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

True